In [1]:
import openai
import pymysql
import time

In [2]:
# Set up Azure OpenAI API key and endpoint
openai.api_key = "EJgyDSQSrrt8FyS9qCoXWExkJsALhSg77kWL9qt6L592RYCyGIw4JQQJ99BBAC77bzfXJ3w3AAABACOG01wO"
openai.api_base = "https://ai-pocs.openai.azure.com/openai/deployments/gpt-4/chat/completions?api-version=2024-08-01-preview"

In [ ]:
# Function to get a connection to MySQL database
def get_db_connection():
    connection = pymysql.connect(
        host="localhost", 
        user="root",  
        password="@DRsri1212", 
        database="MYSQ-DB"  
    )
    return connection

In [4]:
def get_schema_info(db_connection):
    cursor = db_connection.cursor()
    cursor.execute("SHOW TABLES;")
    tables = cursor.fetchall()
    
    schema_info = {}

    # Loop through each table and get column details
    for table in tables:
        table_name = table[0]
        cursor.execute(f"DESCRIBE {table_name};")  # Get columns for each table
        columns = cursor.fetchall()
        schema_info[table_name] = [column[0] for column in columns]  # Collect column names
    
    cursor.close()
    return schema_info

In [5]:
# Function to generate SQL query from natural language question
def generate_sql_query(schema_info, question):
    # Create a prompt for the AI to generate a SQL query
    prompt = f"""
    You are a SQL expert. Based on the following database schema, generate a SQL query to answer the user's question.

    Database Schema:
    """
    
    # Dynamically add the schema information (tables and columns)
    for table, columns in schema_info.items():
        prompt += f"\n{table} ("
        prompt += ", ".join(columns)
        prompt += ")"
    
    prompt += f"\n\nUser Question: {question}\n\nSQL Query:"
    response = openai.Completion.create(
        engine="gpt-4", 
        prompt=prompt,
        max_tokens=150,  
        temperature=0,  
    )
    
    # Extract the SQL query from the response
    sql_query = response.choices[0].text.strip()
    return sql_query

In [6]:
# Function to run SQL query on the database
def run_sql_query(db_connection, query):
    cursor = db_connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    cursor.close()
    return result

In [7]:
# Function to process and answer user queries using the Azure OpenAI model
def process_query(query):
    # Connect to the database
    db_connection = get_db_connection()

    # Retrieve the schema information from the database
    schema_info = get_schema_info(db_connection)

    # Measure time for SQL query generation
    start_time = time.time()

    # Generate SQL query using OpenAI
    sql_query = generate_sql_query(schema_info, query)

    end_time = time.time()
    generation_time = end_time - start_time
    print(f"Time taken to generate SQL query: {generation_time} seconds")

    print(f"Generated SQL Query: {sql_query}")

    # Execute the generated SQL query on the database
    try:
        result = run_sql_query(db_connection, sql_query)
        return result
    except Exception as e:
        return f"Error executing query: {e}"


In [8]:
# Main function to interact with the user
def main():
    while True:
        # Take user input as a question in natural language
        question = input("Please enter your question (or type 'exit' to quit): ")

        if question.lower() == 'exit':
            break

        # Process the query and get the results
        result = process_query(question)

        # Return the result to the user
        print(f"Answer: {result}")

if __name__ == "__main__":
    main()


RuntimeError: 'cryptography' package is required for sha256_password or caching_sha2_password auth methods